In [ ]:
%matplotlib inline

In [ ]:
import rasterio
from glob import glob
from matplotlib import pyplot
import numpy as np

In [ ]:
rasters_fp = glob('../datos/MDE/*.tif')

In [ ]:
rasters_fp

In [ ]:
mammal_1 = rasterio.open('../datos/MDE/m015gh1.tif')

In [ ]:
mammal_1.__class__

In [ ]:
mammal_1.read_crs()

In [ ]:
mammal_1.shape

In [ ]:
band1 = mammal_1.read(1)

In [ ]:
mammal_1.nodata

In [ ]:
band1.__class__

In [ ]:
pyplot.imshow(np.ma.masked_equal(band1, mammal_1.nodata), cmap='PiYG')

In [ ]:
mammal_2 = rasterio.open(rasters_fp[1])

In [ ]:
mammal_2.shape

In [ ]:
mammal_2.bounds

In [ ]:
mammal_1.bounds

In [ ]:
mammal_1.shape

In [ ]:
mammal_1.count

In [ ]:
mammal_1_arr = np.ma.masked_equal(mammal_1.read(1), mammal_1.nodata)

In [ ]:
?mammal_1.read

In [ ]:
mammal_1_arr_feo = mammal_1.read(1, masked=True)

In [ ]:
np.unique(mammal_1_arr)

In [ ]:
np.unique(mammal_1_arr_feo)

In [ ]:
mammal_2_arr = np.ma.masked_equal(mammal_2.read(1), mammal_2.nodata)

In [ ]:
pyplot.imshow((mammal_1_arr+mammal_2_arr > 0), cmap='PiYG')

In [ ]:
pyplot.imshow(mammal_1_arr*mammal_2_arr, cmap='PiYG')

In [ ]:
import geopandas as gpd

In [ ]:
estados_fp = '../datos/destdv1gw_c/destdv1cw.shp'

In [ ]:
estados_gdf = gpd.GeoDataFrame.from_file(estados_fp)

In [ ]:
estados_gdf = estados_gdf[['ENTIDAD', 'geometry']]

In [ ]:
estados_gdf.head()

In [ ]:
estados_gdf.crs

In [ ]:
crs = mammal_1.read_crs()

In [ ]:
crs.to_dict()

In [ ]:
estado_geom = estados_gdf[estados_gdf.ENTIDAD == 'DURANGO'].unary_union # Selecciona la geometria de Durango

In [ ]:
estado_geom

In [ ]:
estado_geom.bounds # Bounding coordinates

In [ ]:
window_estado = mammal_1.window(*estado_geom.bounds)

In [ ]:
?mammal_1.window

In [ ]:
window_estado

In [ ]:
distribution_cut = mammal_1.read(indexes=1, window=window_estado, masked=True)

In [ ]:
distribution_cut.shape

In [ ]:
pyplot.imshow(distribution_cut)

In [ ]:
out_transform_estado

In [ ]:
out_transform_estado = mammal_1.window_transform(window_estado)

In [ ]:
from rasterio.features import rasterize

In [ ]:
estado_mask = rasterize([ estado_geom ], out_shape=distribution_cut.shape, transform=out_transform_estado, fill=0, default_value=1)

In [ ]:
?rasterize

In [ ]:
pyplot.imshow(estado_mask, cmap='PiYG')

In [ ]:
pyplot.imshow(estado_mask*distribution_cut, cmap='PiYG')

In [ ]:
import rasterio.tools.mask

In [ ]:
from shapely.geometry import mapping

In [ ]:
estados_AOI = estados_gdf[estados_gdf.ENTIDAD.isin(["DURANGO", "PUEBLA"])]

In [ ]:
estados_AOI = estados_AOI.geometry.apply(mapping).tolist()

In [ ]:
estados_AOI

In [ ]:
out_image, out_transform = rasterio.tools.mask.mask(mammal_1, estados_AOI, crop=True)

In [ ]:
out_image.shape

In [ ]:
pyplot.imshow(np.ma.masked_equal(out_image[0,:,:], 255), cmap='PiYG')